In [1]:
# global constants
g_maxAngVel = pi / 2 # move in azimuth or elevation at max rate of 90 deg / sec
g_maxScanRate = 1000 # scans points at the rate of 1000 pts/sec

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;


# function to simulate a scan from one spherical coord to another
# input: from: spherical coordinate triple (radius, theta, phi) to start from
#        to:   spherical coordinate triple (radius, theta, phi) to end at
# output: n x 3 list of spherical points
function scan_from_to(from, to)
    delta = to .- from
    time = maximum(abs.(delta[2:3])) / g_maxAngVel
    qty = floor(time * g_maxScanRate)
    rgs = linspace.(from, to, qty)
    return [collect(rgs[1]) collect(rgs[2]) collect(rgs[3])]
end

# function to simulate a scan through a list of (spherical) points
# input:  list of points (spherical) in the order to be visited
# output: list of points (spherical) of the simulated scan points
function scan_tour(pts)
    ret = []
    for i = 1:size(pts,1)-1
        line = scan_from_to(pts[i,:], pts[i+1,:])
        ret = [ret; line]
    end
    return ret
end

function test_transforms()
    # load the feature file
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # convert to spherical
    pts_mcs = pcs_to_mcs_xyz(lr_tfm, pts_pcs) # convert feature locations to MCS
    pts_lr = xyz_to_sph(pts_mcs) # convert feature locations to spherical

    # round trip as a test
    pts_mcs_back = sph_to_xyz(pts_lr)
    pts_pcs_back = mcs_to_pcs_xyz(lr_tfm, pts_mcs_back)

    # points should be the same
    diff = pts_pcs - pts_pcs_back
    display(maximum(diff))
end

function test_scan()
    # load data.
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # shuffle an index vector
    idx = Vector(1:size(pts_pcs,1))
    rng = MersenneTwister(1234)
    idx = shuffle(rng, idx)
    
    # shuffle input pts and convert them to spherical
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs[idx,:]))

    # generate scan data
    scan = scan_tour(sph)
    
    # back to pcs
    scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
    println(size(scan))
    
    # save
    writecsv("scan1.csv", scan)
end

test_scan (generic function with 1 method)

In [2]:
# function that computes the travel time from spherical coordinate 'from' to 'to'
# (assumes instantaneous acceleration to max velocity)
travel_time(from, to) = maximum(abs.((to .- from)[2:3])) / g_maxAngVel

# function that takes a list of nx3 spherical coordinates and returns an n x n 
# matrix where A(i,j) where  the time it takes to move from point i to point j
cost_matrix(sph) = [travel_time(sph[i,:],sph[j,:]) for i=1:size(sph,1), j=1:size(sph,1)]


function find_subtour(idxs,start)
    subtour = [start]
    while true
        next = idxs[subtour[end]]
        push!(subtour,next)
        if start == next
            break
        end
    end
    return subtour
end

function find_subtours(idxs)
    remaining = [i for i=1:length(idxs)]
    subtours = []
    while length(remaining) > 0
        subtour = find_subtour(idxs, remaining[1])
        push!(subtours, subtour)
        remaining = setdiff(remaining, subtour)
    end
    return subtours
end

function features_to_cost(features, lr_location)
# xyz location of features
    pts_pcs = convert(Array{Float64, 2}, features[:,2:4]) # feature locations in PCS

    # spherical coordinates with origin at laser radar
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_location, pts_pcs))
    
    n = size(sph,1) # number of features
    cost = cost_matrix(sph) # cost to travel from feature i to feature j
    
    return sph, n, cost
end

features_to_cost (generic function with 1 method)

In [45]:
using JuMP, Gurobi

function min_path(features, lr_location)
    sph, n, cost = features_to_cost(features, lr_location)

    # TSP solve
    m = Model(solver=GurobiSolver(OutputFlag=0, LazyConstraints=1))
    @variable(m, z[1:n, 1:n], Bin)
    @objective(m, Min, sum(z.*cost)) # minimize the sum of movement times
    @constraint(m, c1[i in 1:n], sum(z[i,:]) == 1) # leave each feature once
    @constraint(m, c2[j in 1:n], sum(z[:,j]) == 1) # arrive at each feature once
    @constraint(m, c3[ii in 1:n], z[ii,ii] == 0) # can't transition to self
    @constraint(m, c4[i=1:n,j=1:n], z[i,j] + z[j,i] <= 1)

    solved = false;
    soln = [];
    
    function lazy_subtours(cb)
        soln = getvalue(z)
        # check for subtours
        idxs = [find(soln[i,:] .>= 0.9)[1] for i=1:n]
        subtours = find_subtours(idxs)
        
        if length(subtours) == 1
            solved = true;
        else
            # add constraints to prohibit these subtours
            for subtour in subtours
                len = length(subtour)
                # prohibit these particular features from being a tour
                @lazyconstraint(cb, sum( z[subtour[i],subtour[i+1]] for i = 1:len-1 ) <= len-2)
                # and prohibit the reverse
                @lazyconstraint(cb, sum( z[subtour[i+1],subtour[i]] for i = 1:len-1 ) <= len-2)
            end            
        end        
    end
    
    addlazycallback(m, lazy_subtours)
    
    status =  solve(m)
    println(status)
    if solved
        # display(@sprintf("solved in %d iterations\n", iter))
        # extract feature order
        order = [1 for i=1:n]
        for i in 2:n
            order[i] = find(soln[order[i-1],:] .== 1)[1]
        end
        display(order)

        # simulate a scan of the features
        # scan = scan_tour(sph[order,:])
        ret = []
        for i = 1:n
            to = find(soln[i,:] .== 1)[1]
            @printf("scan from %d to %d\n", i, to)
            line = scan_from_to(sph[i,:], sph[to,:])
            ret = [ret; line]
        end
        # back to pcs
        return mcs_to_pcs_xyz(lr_location, sph_to_xyz(ret))
    end
end




min_path (generic function with 1 method)

Alternative Problem Formulation

In [7]:
function subtour(edges, n)
    visited = [false for _ in 1:length(edges)]
    cycles  = []
    lengths = []
    selected = [[] for _ in 1:n]
    
    for edge in edges
        push!(selected[edge[1]], edge[2])
    end

    while true
        current = findfirst(x -> x == false, visited)
        this_cycle = [current]
        
        while true
            visited[current] = true
            
            neighbors = []
            for x in selected[current]
                if visited[x] == false
                    push!(neighbors, x)
                end
            end
            
            if length(neighbors) == 0
                break
            end
            
            current = neighbors[1]
            push!(this_cycle, current)
        end
        
        push!(cycles, this_cycle)
        push!(lengths, length(this_cycle))
        if(sum(lengths) == n)
            break
        end
    end
    
    return cycles
end

subtour (generic function with 1 method)

In [27]:
using JuMP, Gurobi

function min_path(features, lr_location)
    sph, n, cost = features_to_cost(features, lr_location)

    # TSP solve
    m = Model(solver=GurobiSolver(OutputFlag=0, LazyConstraints=1))
    @variable(m, z[1:n, 1:n], Bin)
    @objective(m, Min, sum(z.*cost) / 2) # minimize the sum of movement times
    @constraint(m, c1[i in 1:n], sum(z[i,:]) == 2) # leave each feature once
    @constraint(m, c11[i in 1:n], sum(z[:,i]) == 2)
    @constraint(m, c2[i in 1:n, j in 1:n], z[i,j] == z[j,i])
    @constraint(m, c3[ii in 1:n], z[ii,ii] == 0) # can't transition to self
    
    solved = false;
    soln = [];
    iteration = 1;
    
    function lazy_subtours(cb)
        iteration = iteration + 1
        soln = getvalue(z)
        # check for subtours
        edges = []
        for i = 1:n
            for j = 1:n
                if soln[i,j] > 0.5
                    push!(edges, [i, j])
                end
            end
        end
        subtours = subtour(edges, n)
        
        if length(subtours) == 1
            println("Solved!")
            solved = true;
            println(subtours)
            return
        else
            # add constraints to prohibit these subtours
            #for this_subtour in subtours
            this_subtour = subtours[1]
            
            len = length(this_subtour)
            # prohibit these particular features from being a tour
            @lazyconstraint(cb, sum( z[this_subtour[i],this_subtour[i+1]] for i = 1:len-1 ) +
                z[this_subtour[len],this_subtour[1]] <= len-1)
            #end
        end
    end
    
    addlazycallback(m, lazy_subtours)
    
    status =  solve(m)
    soln = getvalue(z)
    if solved
        # display(@sprintf("solved in %d iterations\n", iter))
        # extract feature order
#         order = [1 for i=1:n]
#         for i in 2:n
#             order[i] = find(soln[order[i-1],:] .== 1)[1]
#         end
#         display(order)

        # simulate a scan of the features
        # scan = scan_tour(sph[order,:])
        ret = []
        for i = 1:n
#             to = find(soln[i,:] .== 1)[1]
#             @printf("scan from %d to %d\n", i, to)
# #             line = scan_from_to(sph[i,:], sph[to,:])
# #             ret = [ret; line]
            for j = 1:n
                if soln[i,j] > 0.5
                    println("Edge at (", i, ",", j, ")")
                end
            end
        end
        # back to pcs
#         return mcs_to_pcs_xyz(lr_location, sph_to_xyz(ret))
    end
end




min_path (generic function with 1 method)

In [28]:
features = readcsv("50_Holes_FrontLeft.csv");
lr_position = readcsv("Location1.csv"); # transformation matrix for LR's positionend

In [ ]:
scan = @time(min_path(features[1:21,:], lr_position))
println(scan)

#display(size(scan))
#writecsv("scan2.csv", scan)

Academic license - for non-commercial use only
Solved!
Any[[1, 4, 8, 5, 7, 11, 2, 21, 14, 15, 18, 20, 19, 17, 16, 12, 13, 3, 10, 9, 6]]
Solved!
Any[[1, 4, 9, 3, 12, 13, 16, 19, 20, 18, 21, 17, 15, 14, 2, 11, 10, 6, 8, 5, 7]]
Solved!
Any[[1, 5, 4, 8, 13, 3, 12, 15, 14, 17, 2, 21, 18, 20, 19, 16, 11, 10, 9, 7, 6]]
Solved!
Any[[1, 4, 7, 9, 10, 11, 16, 19, 20, 18, 21, 2, 17, 14, 15, 12, 3, 13, 8, 6, 5]]
Solved!
Any[[1, 5, 6, 4, 9, 10, 11, 16, 15, 14, 17, 2, 19, 20, 18, 21, 12, 3, 13, 8, 7]]
Solved!
Any[[1, 5, 6, 4, 9, 10, 11, 16, 15, 14, 17, 2, 19, 20, 18, 21, 12, 3, 13, 8, 7]]
Solved!
Any[[1, 4, 9, 10, 11, 16, 15, 14, 17, 2, 19, 20, 18, 21, 12, 3, 13, 8, 7, 6, 5]]
Solved!
Any[[1, 4, 7, 9, 10, 11, 16, 15, 14, 17, 2, 19, 20, 18, 21, 12, 3, 13, 8, 6, 5]]
Solved!
Any[[1, 7, 5, 8, 4, 6, 9, 13, 3, 12, 2, 21, 18, 20, 19, 16, 15, 14, 17, 11, 10]]
Solved!
Any[[1, 4, 5, 6, 7, 8, 9, 21, 18, 19, 20, 16, 15, 14, 17, 2, 12, 13, 3, 11, 10]]
Solved!
Any[[1, 4, 5, 6, 7, 8, 9, 13, 3, 12, 21, 18, 19, 20, 16